In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [1]:
tickers = ["SPY", "GME", "AMC", "TSLA", "BB", "NVDA", "BBBY", "MVIS", "BABA", "AMD"]
ohlc = yf.download(tickers, period="max")
prices = ohlc["Adj Close"]
prices.tail()

NameError: name 'RBLX' is not defined

In [6]:
prices.head()

,AMC,AMD,BABA,BB,BBBY,GME,MVIS,NVDA,SPY,TSLA
Date,,,,,,,,,,
1980-03-17,NaN,3.145833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-18,NaN,3.031250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-19,NaN,3.041667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-20,NaN,3.010417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-21,NaN,2.916667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
prices = prices[prices.index >= "2017-08-01"]

In [ ]:
prices.to_csv("stock_prices.csv")